In [1]:
import os
os.chdir("../..")

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
from utils.dataset_loader import load_datasets
from utils.model_utils import initialize_model
from utils.train_utils import train_model
from utils.metrics import evaluate_model
from utils.visualization import (
    plot_training, 
    plot_confusion_matrix, 
    plot_precision_recall, 
    plot_roc_curve
)

C:\Users\Yash\Documents\Development\Projects\final_year\env\Lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.23 (you have 1.4.22). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[INFO] Using device: {device}")

[INFO] Using device: cuda


In [4]:
# Model Name (Change this to test other models)
model_name = "efficientnet_b0"

In [5]:
# Directories
os.makedirs("outputs/models", exist_ok=True)
os.makedirs("outputs/plots", exist_ok=True)

In [6]:
# Load dataset with augmentation
print("[INFO] Loading augmented datasets...")
data_dir = "wildfire_dataset_scaled"
batch_size = 32
train_loader, val_loader, test_loader = load_datasets(
    data_dir, batch_size=batch_size, augmentation="augmented"
)
print("[INFO] Datasets loaded successfully!")


[INFO] Loading augmented datasets...
[INFO] Datasets loaded successfully!


C:\Users\Yash\Documents\Development\Projects\final_year\env\Lib\site-packages\pydantic\main.py:214: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


In [7]:
def objective(trial):
    print(f"[DEBUG] Starting trial {trial.number}...")

    # Hyperparameter suggestions
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    dropout_prob = trial.suggest_float("dropout_prob", 0.2, 0.5)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD", "AdamW"])

    print(f"[DEBUG] Suggested hyperparameters: lr={lr}, weight_decay={weight_decay}, optimizer={optimizer_name}")

    # Initialize model
    model = initialize_model(
        model_name=model_name,
        num_classes=2,
        pretrained=True,
        freeze_all=False,
        dropout_prob=dropout_prob
    )
    model.to(device)

    # Optimizer setup
    if optimizer_name == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_name == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
    elif optimizer_name == "AdamW":
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

    # Scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", patience=3, factor=0.5)

    # Loss function
    criterion = nn.CrossEntropyLoss()

    # Train the model
    print("[INFO] Starting model training...")
    save_path = f"outputs/models/{model_name}_trial_best.pth"
    history = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        save_path=save_path,
        early_stop_patience=5,
        monitor_metric="val_recall",
    )
    print("[INFO] Training completed!")

    # Return validation recall
    val_recall = history["val_recall"][-1]
    print(f"[INFO] Trial {trial.number} - Final Validation Recall: {val_recall:.4f}")
    return val_recall


In [8]:
# Run Optuna Hyperparameter Tuning
print("[INFO] Running hyperparameter optimization...")
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)
print(f"[INFO] Best parameters: {study.best_params}")

# Train the final model with the best parameters
print("[INFO] Training final model with best hyperparameters...")
final_model = initialize_model(
    model_name=model_name,
    num_classes=2,
    pretrained=True,
    freeze_all=False,  # All layers unfrozen
    dropout_prob=study.best_params["dropout_prob"]
)
final_model.to(device)

[I 2024-12-18 23:22:56,049] A new study created in memory with name: no-name-ef5e770b-95de-450e-8835-91b7ce282d54


[INFO] Running hyperparameter optimization...
[DEBUG] Starting trial 0...
[DEBUG] Suggested hyperparameters: lr=5.943545145687012e-05, weight_decay=4.7694576162678014e-05, optimizer=Adam
[INFO] Starting model training...

Starting training...



[INFO] Learning rate adjusted to: 0.000059
[INFO] Best model saved with val_recall: 0.7805
Epoch [1]: Train Loss: 0.6437, Train Acc: 0.6259 | Val Loss: 0.5507, Val Acc: 0.7388, Val Recall: 0.7805, Val F1: 0.7853


[INFO] Learning rate adjusted to: 0.000059
[INFO] Best model saved with val_recall: 0.8374
Epoch [2]: Train Loss: 0.5025, Train Acc: 0.7668 | Val Loss: 0.4360, Val Acc: 0.7687, Val Recall: 0.8374, Val F1: 0.8158


[INFO] Learning rate adjusted to: 0.000059
[INFO] Best model saved with val_recall: 0.8699
Epoch [3]: Train Loss: 0.4061, Train Acc: 0.8177 | Val Loss: 0.3874, Val Acc: 0.8259, Val Recall: 0.8699, Val F1: 0.8594


...

[INFO] Learning rate adjusted to: 0.000029
[INFO] Best model saved with val_recall: 0.7317
Epoch [1]: Train Loss: 0.6762, Train Acc: 0.5612 | Val Loss: 0.6270, Val Acc: 0.6716, Val Recall: 0.7317, Val F1: 0.7317


[INFO] Learning rate adjusted to: 0.000029
[INFO] Best model saved with val_recall: 0.8130
Epoch [2]: Train Loss: 0.5901, Train Acc: 0.7038 | Val Loss: 0.5545, Val Acc: 0.7438, Val Recall: 0.8130, Val F1: 0.7952


[INFO] Learning rate adjusted to: 0.000029
[INFO] Best model saved with val_recall: 0.8333
Epoch [3]: Train Loss: 0.5129, Train Acc: 0.7488 | Val Loss: 0.4780, Val Acc: 0.7811, Val Recall: 0.8333, Val F1: 0.8233


[W 2024-12-18 23:44:43,568] Trial 6 failed with parameters: {'lr': 2.8896862245871168e-05, 'weight_decay': 0.0008640904040761108, 'dropout_prob': 0.3581145510166829, 'optimizer': 'Adam'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\Yash\Documents\Development\Projects\final_year\env\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\Yash\AppData\Local\Temp\ipykernel_15596\468884178.py", line 39, in objective
    history = train_model(
              ^^^^^^^^^^^^
  File "C:\Users\Yash\Documents\Development\Projects\final_year\utils\train_utils.py", line 69, in train_model
    train_loss += loss.item() * images.size(0)
                  ^^^^^^^^^^^
KeyboardInterrupt
[W 2024-12-18 23:44:43,569] Trial 6 failed with value None.


KeyboardInterrupt: 

In [ ]:
# Optimizer and Scheduler
optimizer = optim.Adam(
    final_model.parameters(), 
    lr=study.best_params["lr"], 
    weight_decay=study.best_params["weight_decay"]
)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", patience=3, factor=0.5)

# Loss Function
criterion = nn.CrossEntropyLoss()

In [ ]:
# Final Training
final_save_path = f"outputs/models/{model_name}_final.pth"
history = train_model(
    model=final_model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    device=device,
    save_path=final_save_path,
    early_stop_patience=5,
    monitor_metric="val_recall",
    gradient_clip_val=1.0,  # Gradient Clipping
)

In [ ]:
# Final Evaluation and Visualization
print("[INFO] Evaluating final model...")
metrics = evaluate_model(final_model, test_loader, ["No Fire", "Fire"], device)

In [ ]:
# Plots
plot_training(history, f"outputs/plots/{model_name}_training_curve.png")
plot_confusion_matrix(
    metrics["confusion_matrix"], 
    ["No Fire", "Fire"], 
    output_path=f"outputs/plots/{model_name}_confusion_matrix.png"
)
if "y_true" in metrics and "y_probs" in metrics:
    y_true = metrics["y_true"]
    y_probs = [prob[1] for prob in metrics["y_probs"]]
    plot_precision_recall(y_true, y_probs, f"outputs/plots/{model_name}_precision_recall_curve.png")
    plot_roc_curve(y_true, y_probs, f"outputs/plots/{model_name}_roc_curve.png")

print("[INFO] Final results saved successfully!")